In [1]:
from datetime import datetime
from concurrent import futures

import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import csv

def download_stock(stock):
    """ try to query the iex for a stock, if failed note with print """
    try:
        print(stock)
        #stock_df = web.DataReader(stock,'iex', start_time, now_time)        
        stock_df = web.DataReader('WIKI/' + stock,'quandl', '2011-09-27', '2013-03-04') #Try Quandl instead.
        stock_df['Name'] = stock
        output_name = stock + '_data.csv'
        stock_df.to_csv(output_name)
    except:
        bad_names.append(stock)
        print('bad: %s' % (stock))

if __name__ == '__main__':

    """ set the download window """
    now_time = datetime.now()
    #start_time = datetime(now_time.year - 5, now_time.month , now_time.day)
    start_time = datetime(2011,9,27)

    """ list of s_anp_p companies """
    s_and_p = []
    with open('SP500.csv') as SP500csv:
        reader = csv.DictReader(SP500csv)
        for row in reader:
            s_and_p.append(row['Symbol'])
        
    bad_names =[] #to keep track of failed queries

    """here we use the concurrent.futures module's ThreadPoolExecutor
        to speed up the downloads buy doing them in parallel 
        as opposed to sequentially """

    #set the maximum thread number
    max_workers = 50

    workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_stock, s_and_p)


    """ Save failed queries to a text file to retry """
    if len(bad_names) > 0:
        with open('failed_queries.txt','w') as outfile:
            for name in bad_names:
                outfile.write(name+'\n')

    #timing:
    finish_time = datetime.now()
    duration = finish_time - now_time
    minutes, seconds = divmod(duration.seconds, 60)
    print('getSandP_threaded.py')
    print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
    #The threaded script took 0 minutes and 31 seconds to run.